## 1. Import Libraries







In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re, unicodedata
import html

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_colwidth', None)

## 2. Importation des données

In [2]:
fightclub_df = pd.read_csv('fightclub_critiques.csv')
interstellar_df = pd.read_csv('interstellar_critique.csv')

fightclub_df["film"] = "Fight Club"
interstellar_df["film"] = "Interstellar"

data = pd.concat([fightclub_df, interstellar_df], ignore_index = True)

In [9]:
#data.head()

In [4]:
# Appercu rapide sur la data
print(data.shape)
print(data.info())

(2000, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       2000 non-null   int64 
 1   URL                      2000 non-null   object
 2   rating                   2000 non-null   int64 
 3   review_date_creation     2000 non-null   object
 4   review_date_last_update  2000 non-null   object
 5   review_hits              2000 non-null   int64 
 6   review_content           1999 non-null   object
 7   review_title             1481 non-null   object
 8   gen_review_like_count    2000 non-null   int64 
 9   username                 2000 non-null   object
 10  user_id                  2000 non-null   int64 
 11  film                     2000 non-null   object
dtypes: int64(5), object(7)
memory usage: 187.6+ KB
None


## 3. EDA Light


In [8]:
print(data.isna().sum())
print("_Duplicates rows :", data.duplicated().sum())
#print(data[["rating", "review_hits", "gen_review_like_count"]].describe())

id                           0
URL                          0
rating                       0
review_date_creation         0
review_date_last_update      0
review_hits                  0
review_content               1
review_title               519
gen_review_like_count        0
username                     0
user_id                      0
film                         0
dtype: int64
_Duplicates rows : 0


- Aucun doublon nest présent dans le dataset
- La colonne _review_title_ contient 519 valeurs manquantes, soit un nombre significatif

## 4. Nettoyage et préparation de données

####  4.1 Suppresion des NaN

Pour ce projet, le texte d'entrée utilisé pour le modele d'embedding suit le format : _title.review_

Pour garantir la cohérence de ce format, j'ai décidé de supprimer toutes les lignes ou le titre ou le contenu de la critique est manquant

In [10]:
data = data.dropna(subset=['review_content', 'review_title']).reset_index()

#### 4.2 préparation des données textuelles



Afin de garantir la cohérence et la qualité linguistiques de textes, j'ai effectué une preparation textuelle sur les colonnes
_review_title_ et _review_content_. Cette étape vise à :   

*   Supprimer les balises HTML
*   Supprimer la ponctuation
*   Uniformiser les espaces et les caractères accentués
*   Combiner le titre et le texte : titre.critique
*   Mettre le texte en minuscules


**Note :** Utilisation de l'IA dans cette partie pour trouver les regex appropriées et le syntaxe pour supprimer des balises HTML




In [11]:
def clean_text(title, html):

    """
    Fonction qui nettoie et normalise le texte d'un titre et le contenu d'une critique

    paramètres :

    titre (str): associé à la critique
    html (str): le contenu HTML de la critique

    Retourne : le texte nettoyé, normalisé et concaténé (str)

    """
    cleaned_review = BeautifulSoup(str(html), "html.parser").get_text(" ")
    cleaned_review = re.sub(r"[^\w\s]", " ", cleaned_review)

    title = str(title).strip()
    title = re.sub(r"[^\w\s]", " ", title)
    title = re.sub(r"\s+$", "", title)

    title_review = f"{title}. {cleaned_review}".strip()

    title_review = unicodedata.normalize("NFKC", title_review)
    title_review = re.sub(r"\s+", " ", title_review).strip().lower()

    return title_review

In [12]:
data['review_clean'] = data.apply(lambda row: clean_text(row["review_title"], row["review_content"]), axis=1)

In [14]:
#data.head()
#data["review_content"][1476]
#data["review_title"][1477]
data["review_clean"][2]

'sons of anarchy. qu une oeuvre aussi folle aussi inconfortable aussi ambigüe aussi inclassable que fight club sorte d un gros studio aussi conservateur que la fox reste une des blagues les plus brillantes de cette fin de siècle cinquante millions de dollars une méga star so sexy en tête d affiche un traitement de blockbuster tout ça au service d une adaptation du roman destroy de chuck palahniuk aux commandes de cette entreprise casse gueule et prête à exploser à la moindre anicroche david fincher ravi de prendre la place laissée vacante par peter jackson bryan singer et danny boyle donne corps aux mots de palahniuk avec un brio certain virevoltante clinquante ludique presque sans limite la mise en scène du cinéaste joue magistralement avec notre perception avec notre complicité explose le quatrième mur pour nous faire ressentir à plein nez le parcours de son personnage principal interprété avec talent par edward norton enfants du consumérisme d une course effrénée à la réussite et au

## 5. Tokenization et Vectorisation (embedding)

Cette partie vise à transformer les critiques textuelles en représentations numériques (embeddings) pour mesurer leur similarité sémantique.

- J'ai choisi les modèles récents comme _Sentence-Transformer E5_, _OpenAI_ ou _Gemini_ car ils permettent de réaliser automatiquement la tokenization et produisent des embeddigns riches capables de comprendre le sens complet d'une phrase.

- Contrairement aux anciens modeles (comme _Word2Vec_ ou _TF-IDF_), qui nécessitent une tokenization manuelle et ne prennent pas en compte le contexte global du texte

- En plus, les modeles modernes sont multilingues, ce qui est un grand avantage pour ce projet, car certains titres de critiques sont en anglais


Pour ce projet, j'ai  choisi le modele **E5**, une version améliorée de SBERT car il est  :
   - Gratuit et exécutable localement
   - Rapide et optimisé
   - multilingue
   - et puissant sur le plan sématique




In [15]:
model = SentenceTransformer("intfloat/multilingual-e5-base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Ce modèle a été entraîné avec un format spécial d’entrée :

- Les textes doivent commencer par le préfixe "passage: "
- Les requêtes (la critique qu’on cherche à comparer) doivent commencer par "query:"
- Afin de comparer uniquement le sens des critiques, pas leur longueur, chaque embedding de review est normalisé grace à _normalize_embeddings=True_

In [17]:
review_texts = data["review_clean"].tolist()
review_texts = ["passage: " + t for t in review_texts]
embeddings = model.encode(review_texts, normalize_embeddings=True, show_progress_bar=True)

data["embeddings"] = list(embeddings)

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

## 6. Test du sytème de recommandation

In [24]:
def recommander_critique_similaire(film, top_n=5):

  """
  Recommande les critiques similaires à une critique lue par l'utlisateur
  """

  critique_lu = input(str("Entrer la critique lu :"))

  data_film = data[data["film"] == film].reset_index(drop=True)
  embeddings_film = np.vstack(data_film["embeddings"].values)

  critique_emb = model.encode(["query: " + str(critique_lu)], normalize_embeddings=True)

  simalarity = cosine_similarity(critique_emb, embeddings_film)[0]
  top_idx = np.argsort(simalarity)[::-1][:top_n]

  cols = ["film", "rating", "username", "review_title", "review_content"]
  return data_film.iloc[top_idx][cols]

- **Note :** l'IA est utilisé dans la fonction _formater_critiques_simailaire_ afin de trouver les bon regex pour nettoyer les critiques similaires et retourner un texte clair et lisible pour utlisateur

In [39]:
def formater_critiques_simailaire(text):

  """
  Nettoie le texte d'une critique pour un affichage propore

  """

  txt = BeautifulSoup(text, "html.parser").get_text(" ")
  txt = html.unescape(txt)
  txt = re.sub(r"[\x00-\x1f\x7f-\x9f]", " ", txt)
  txt = re.sub(r"[?!]{2,}", " ", txt)     # ??, !!! → un seul espace
  txt = re.sub(r"(\?\s*){2,}", " ", txt)  # ? ? ? → un seul espace
  txt = re.sub(r"[^\w\s.,;:!?\'’”“€éèàùêôîïç❤️☺️☹️👍👏😍😅😭😡😬😕]", " ", txt)
  txt = re.sub(r"\s+", " ", txt).strip()

  return txt


In [38]:
film = "Fight Club"
top_n = 10
# critique = "Je n'ai pas aimé le film, trop de bagarres à mains nues"
# critique = "J'ai beaucoup aimé le film"


# test du systeme de recommandation
res = recommander_critique_similaire(film, top_n=top_n)['review_content']
for txt in res:
  print("\n", formater_critiques_simailaire(txt))

Entrer la critique lu :Je n'ai pas aimé le film, trop de bagarres à mains nues

 Les tours qui explose et le delire anarchiste aussi c’etait genant

 Je déconne, j'ai pas tant apprécié que ça. En plus c'est bourré de ☺️ ️ ️ ☹️

 Si vous n'aimez pas les gens qui se tapent dessus sans raison et le sang épargnez vous ce film. Chute prévisible.

 Je n'ai pas pu visionner ce film en entier. J'ai arrêté au bout d'environ 10 mn... Je dois être une âme sensible.

 J’ai une chose à dire: INCROYABLE. J’ai vraiment trop trop aimé, je ne m’attendais pas du tout à tous les rebondissements qu’il y a pu avoir dans le film. J’ai trouvé que le synopsis n’était pas du tout à la hauteur du film en lui même!

 Je ne comprends pas l'engouement des gens et leur volonté de me dire t'as pas vu fight club, regarde le c'est génial . Je me suis ennuyé au point où le twist final ne m'a rien fait. Je n'ai même pas vu une quelconque morale ou message. La vous allez le dire que je dois le regarder une 2e fois et je 